In [ ]:
from IPython.display import display, HTML, clear_output
from openai import OpenAI

import json
import ipywidgets as widgets
import os

# api_key=""

In [ ]:
system_prompt = """You are an AI agent. You can only do the following tasks:
1. Modify a text file.

You can only output a json with the following format:
{
    "action": "modify_file",
    "file_path": "path/to/file.txt",
    "content": "new content"
}
In the action field, it can only have the following values:
1. modify_file
2. no_action

Let me give you some examples:

Example 1:
User input: "Add abcde to the file test.txt"
In this example, user want to modify a file, so, you should output:
{
    "action": "modify_file",
    "file_path": "test.txt",
    "content": "abded"
}

Example 2:
User input: "Write a one-sentence bedtime story about a unicorn"
In this example, user does not want to modify a file, so, you should output:
{
    "action": "no_action",
    "reason": "I cannot understand the user's input"
}

If you cannot understand the user's input, you can output:
{
    "action": "no_action",
    "reason": "I cannot understand the user's input"
}

Now, I give you the user input, please generate output accordingly.
Here is the user input: 
"""

In [ ]:
def modify_file(file_path, content):
    with open(file_path, 'a') as f:
        f.write(content)

def llm(user_prompt):
    client = OpenAI()
    response = client.responses.create(
        model="gpt-5-nano",
        input=system_prompt + " " + user_prompt
    )
    return response.output_text

def handle_task(task_in_str):
    task_in_json = json.loads(task_in_str)
    if task_in_json['action'] == 'modify_file':
        modify_file(task_in_json['file_path'], task_in_json['content'])

In [12]:
def create_interactive_interface():
    # Create input widget
    text_input = widgets.Text(
        value='',
        placeholder='Enter text here',
        description='Input:',
        style={'description_width': 'initial'}
    )

    button_1 = widgets.ToggleButton(
        description='Earning',
        button_style='primary',
        layout=widgets.Layout(width='100px')
    )
    button_2 = widgets.ToggleButton(
        description='Stock Price',
        button_style='primary',
        layout=widgets.Layout(width='100px')
    )

    def on_toggle(change):
        if change['owner'] == button_1 and change['new']:
            button_2.value = False
        elif change['owner'] == button_2 and change['new']:
            button_1.value = False
    button_1.observe(on_toggle, names='value')
    button_2.observe(on_toggle, names='value')
    button_box = widgets.HBox([button_1, button_2])
    
    # Create output area
    output_area = widgets.Output()
    
    def on_button_click(b):
        print(button_1.value)
        print(button_2.value)
        with output_area:
            clear_output(wait=True)
            user_input = text_input.value
            response = llm(user_input)
            handle_task(response)
            print(f"Result: {response}")
    button = widgets.Button(
        description='Submit',
        button_style='primary',
        layout=widgets.Layout(width='100px')
    )
    button.on_click(on_button_click)
    
    # Display the interface
    display(widgets.VBox([
        widgets.HTML("<h1>AI Agent</h1><p></p>"),
        button_box,
        text_input,
        button,
        output_area
    ]))

create_interactive_interface()